In [8]:
import numpy as np
from sklearn.metrics import mean_squared_error

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

# 실제 R 행렬과 예측 행렬의 오차를 구하는 함수
def calculate_rmse(R, P, Q, non_zeros):
    error = 0

    full_pred_matrix = np.dot(P, Q.T)

    # 여기서 non_zeros는 아래 함수에서 확인할 수 있다.
    x_non_zero_ind = [non_zeros[0] for non_zeros in non_zeros]
    y_non_zero_ind = [non_zeros[1] for non_zeros in non_zeros]

    # 원 행렬 R에서 0이 아닌 값들만 추출한다.
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    # 예측 행렬에서 원 행렬 R에서 0이 아닌 위치의 값들만 추출하여 저장한다.
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse



def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape

    np.random.seed(1)
    P = np.random.normal(scale=1.0/K, size=(num_users, K))
    Q = np.random.normal(scale=1.0/K, size=(num_items, K))

    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장한다.
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users)
                  for j in range(num_items) if R[i, j] > 0 ]

    # SGD 기법으로 P, Q 매트릭스를 업데이트 함
    for step in range(steps):
        for i, j, r in non_zeros:
            # 잔차 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)

            # Regulation을 반영한 SGD 업데이터 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

        rmse = calculate_rmse(R, P, Q, non_zeros)
        if step % 10 == 0:
            print("iter step: {0}, rmse: {1:4f}".format(step, rmse))

    return P, Q

P, Q = matrix_factorization(R, K=8)
pred_matrix = np.dot(P, Q.T)
print(pred_matrix)


iter step: 0, rmse: 3.307350
iter step: 10, rmse: 3.220914
iter step: 20, rmse: 2.948440
iter step: 30, rmse: 2.170918
iter step: 40, rmse: 1.235560
iter step: 50, rmse: 0.737789
iter step: 60, rmse: 0.516037
iter step: 70, rmse: 0.391679
iter step: 80, rmse: 0.304491
iter step: 90, rmse: 0.237725
iter step: 100, rmse: 0.185911
iter step: 110, rmse: 0.146072
iter step: 120, rmse: 0.115760
iter step: 130, rmse: 0.092807
iter step: 140, rmse: 0.075397
iter step: 150, rmse: 0.062103
iter step: 160, rmse: 0.051858
iter step: 170, rmse: 0.043889
iter step: 180, rmse: 0.037644
iter step: 190, rmse: 0.032724
[[ 4.00697358  2.47762458  1.05913753  1.96899626  0.34643477]
 [ 5.48843366  4.94563205  1.95029548  3.03359291  1.00850497]
 [ 3.14345587  4.62577565  2.99483816  3.97343594  3.98222382]
 [ 4.96268858  2.04217116  0.99114277  1.97241331 -0.29152874]]


In [9]:
print(max(pred_matrix[0]))

4.006973584507223


In [10]:
print(list(pred_matrix[0]).index(max(pred_matrix[0])))

0
